This notebook plots the output of the script `run_keypointreco.py`.

It takes in:
 * the larmatch output larlite file that stores the output of the larmatch network
 * the keypoint output larlite file that stores the output of run_keypointreco.py
 * optionally, if the mcinfo or dlmerged files is provided that contains truth meta data, then one can also plot the true trajectories of particles

In [ ]:
from __future__ import print_function
import os,sys
import chart_studio as cs
import chart_studio.plotly as py
import plotly.graph_objects as go
import numpy as np
import torch
from larlite import larlite
import lardly
from lardly import DetectorOutline

In [ ]:
# load utility to draw TPC outline
detdata = DetectorOutline()

# define some colors for particle types
kp_color_array = np.array( ((255,0,0),    # nu
                            (0,255,0),    # track-start
                            (0,0,255),    # track-end
                            (255,0,255),  # shower
                            (0,255,255),  # shower-michel
                            (255,255,0)),     # shower-delta
                            dtype=np.float64 ) 

In [ ]:
PLOT_MC = True

In [ ]:
# file lists

# The input file
#larmatch_files = ["../testdata/mcc9_v29e_dl_run3b_bnb_nu_overlay_nocrtremerge/larmatchme_dlgen2.1_merged_dlreco_3ab3a878-3aea-4bc7-926e-bbb7554875f1_larlite.root"]
#kpreco_files = ["test_kpreco.root"]
#mcinfo_files = ["../testdata/mcc9_v29e_dl_run3b_bnb_nu_overlay_nocrtremerge/merged_dlreco_3ab3a878-3aea-4bc7-926e-bbb7554875f1.root"]

larmatch_files = ["/home/twongjirad/working/larbys/ubdl/testdata/pi0select_bnb_nu/larmatchme_fileid0000_0020d30b-8b70-4b75-8841-dec0d65feb3f_merged_larlite.root"]
kpreco_files = ["test_pi0.root"]
mcinfo_files = ["/home/twongjirad/working/larbys/ubdl/testdata/pi0select_bnb_nu/pi0select_0020d30b-8b70-4b75-8841-dec0d65feb3f_merged.root"]

In [ ]:
io = larlite.storage_manager( larlite.storage_manager.kREAD )
for f in larmatch_files:
    io.add_in_filename( f )
for f in kpreco_files:
    io.add_in_filename( f )
for f in mcinfo_files:
    io.add_in_filename( f )
io.open()

In [ ]:
# Get entry and relevant data
ENTRY = 0
io.go_to(ENTRY)

ev_larmatch_hits = io.get_data( larlite.data.kLArFlow3DHit, "larmatch" )
ev_keypoints = io.get_data( larlite.data.kLArFlow3DHit, "keypoint" )

npts = ev_keypoints.size()
print("Number of Reco Keypoints: ",npts)
kp_counts = [0,0,0,0,0,0]
for i in range(npts):
    kphit = ev_keypoints.at(i)
    # data in the hit
    # [0-2]: x,y,z
    # [3]: keypoint type
    # [4]: score
    kptype = int(kphit[3])
    kp_counts[kptype] += 1
print("Counts: ",kp_counts)
    
kp_data = {}
for kptype in range(6):
    if kp_counts[kptype]==0:
        continue
    xyz = np.zeros( (kp_counts[kptype], 3), np.float32 )
    kp_data[kptype] = xyz
    
kp_counts = [0,0,0,0,0,0]

for i in range(npts):
    kphit = ev_keypoints.at(i)
    kptype = int(kphit[3])
    xyz = kp_data[kptype]
    for v in range(3):
        xyz[kp_counts[kptype],v] = kphit[v]
    kp_counts[kptype] += 1





In [ ]:
# Plot Reconstructed Keypoints
detlines = detdata.getlines(color=(10,10,10))
score_index = 17 # nu
#score_index = 18 # track-start
#score_index = 19 # track-end
#score_index = 20 # shower-start
lfplot = lardly.data.visualize_larlite_larflowhits( ev_larmatch_hits, "larmatch", score_threshold=0.5, score_index=score_index)
plot_traces = detlines+[lfplot]

if PLOT_MC:
    # Get MC info
    mcinfoplots = lardly.data.visualize_nu_interaction(io, do_sce_correction=True )
    plot_traces += mcinfoplots

for kptype in [5,4,3,2,1,0]:
    if kptype not in kp_data:
        continue
    rgb = kp_color_array[kptype]
    xyz = kp_data[kptype]
    zrgb = "rgb(%d,%d,%d)"%(rgb[0],rgb[1],rgb[2])
    plot_trace = {
        "type":"scatter3d",
        "x": xyz[:,0],
        "y": xyz[:,1],
        "z": xyz[:,2],
        "mode":"markers",
        "name":"KP[%d]"%(kptype),
        "marker":{"color":zrgb,"size":5,"opacity":0.8},
    }
    #print(plot_trace)
    if kptype==0:
        plot_traces += [plot_trace]
    #break


# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='Predicted Keypoints',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": -2, "y": 0.25, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=plot_traces, layout=layout)
fig.show()

In [ ]:
# NOTES
# Probably need to do a DB-scan and remove detached small, low-charge sub-clusters. [DONE!] 
# I suspect these can be used to build a unique code to overtrain on. [might not be the cause -- but definitely saw bad overtraining]
# Plot the space-charge corrected track to verify charge cluster
# several deposits start at x>250 -- weird. i had cut on true position already 